# Day 3: Datetimes & Outliers

**Learning Objectives:**
- Parse datetime columns safely using `pd.to_datetime(..., errors="coerce")`
- Add time parts (date/month/day-of-week/hour) for grouping
- Identify outliers using percentiles / IQR and choose a safe policy
- Use core pandas ops for real work (`.loc`, `.assign`, `groupby/agg`)
- Join tables safely with `merge(validate=...)` and detect join explosions

**Run the two setup cells below before starting.**

## 👤 STEP 1: Enter Your Name (REQUIRED)

In [ ]:
#@title 👤 STEP 1: Enter Your Name (REQUIRED)
#@markdown Enter your name below and run this cell.

FIRST_NAME = ""  #@param {type:"string"}
LAST_NAME = ""  #@param {type:"string"}

FIRST_NAME = FIRST_NAME.strip() or input("First Name: ").strip()
LAST_NAME = LAST_NAME.strip() or input("Last Name: ").strip()

import re
print("=" * 40)
_first = FIRST_NAME.strip()
_last = LAST_NAME.strip()
if len(_first) < 2 or len(_last) < 2:
    raise ValueError("❌ Name too short! Edit the fields above and run this cell again.")
if not re.match(r"^[A-Za-z\s\-]+$", _first) or not re.match(r"^[A-Za-z\s\-]+$", _last):
    raise ValueError("❌ English letters only! Edit the fields above and run this cell again.")
STUDENT_NAME = f"{_first.upper()} {_last.upper()}"
print(f"✅ Welcome, {STUDENT_NAME}!")

ValueError: ❌ Name too short! Edit the fields above and run this cell again.

## ⚙️ STEP 2: Setup (REQUIRED)

In [ ]:
#@title ⚙️ STEP 2: Setup (REQUIRED)
#@markdown Run this cell to install dependencies and set up telemetry.

try: STUDENT_NAME
except NameError: raise ValueError("❌ Run the cell above first!")

# Install dependencies (works in Colab and local Jupyter)
import subprocess
import sys
subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", "posthog"])

import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import random
import hashlib
import time
import uuid
import posthog
from functools import wraps

# For reproducibility
np.random.seed(42)
random.seed(42)

# === TELEMETRY SETUP ===
_H, _K = "https://us.i.posthog.com", "phc_I7GaZ4p1Ox5PRM5egRrHFxz3ZCdh3zyKQ7B6jxJOWis"
_NB, _SESS = "D3_Datetimes_Outliers", uuid.uuid4().hex[:8]
_SID = hashlib.md5(f"{STUDENT_NAME}-bootcamp".encode()).hexdigest()[:12]
posthog.project_api_key, posthog.host = _K, _H
_attempts = {}
_cell_num = 0

def _capture(e, p=None):
    """Send event to PostHog."""
    try: posthog.capture(_SID, e, {"nb": _NB, "sess": _SESS, "student": STUDENT_NAME, **(p or {})})
    except: pass

def _cell(cell_id, code_preview=""):
    """Track cell execution. Call at START of every code cell."""
    global _cell_num
    _cell_num += 1
    _capture("cell", {"cell_id": cell_id, "cell_num": _cell_num, "code": code_preview[:500]})

def tracked_test(ex):
    """Decorator to track test function results."""
    def d(f):
        @wraps(f)
        def w(*a,**k):
            _attempts[ex] = _attempts.get(ex,0)+1
            t,ok,err = time.time(),False,None
            try: r=f(*a,**k); ok=True; return r
            except Exception as x: err=str(x)[:200]; raise
            finally: _capture("test", {"ex":ex,"ok":ok,"try":_attempts[ex],"ms":int((time.time()-t)*1000),"err":err})
        return w
    return d

_capture("start")
print(f"✅ Ready! {STUDENT_NAME} | Session: {_SESS}")

## Generate Sample Data

We'll create sample `orders` and `users` DataFrames for the exercises.

In [ ]:
#@title Generate Sample Data
_cell("generate_data")

def generate_sample_data():
    """Generate sample orders and users data for exercises."""

    # Generate users
    n_users = 50
    countries = ["USA", "UK", "Germany", "France", "Japan", "Australia"]
    users = pd.DataFrame({
        "user_id": [f"U{i:04d}" for i in range(1, n_users + 1)],
        "country": np.random.choice(countries, n_users),
        "signup_date": pd.date_range("2024-01-01", periods=n_users, freq="3D").astype(str),
    })

    # Generate orders
    n_orders = 500
    statuses = ["paid", "paid", "paid", "paid", "refund", "pending"]  # weighted

    # Create some intentionally messy datetime formats
    base_dates = pd.date_range("2024-06-01", periods=n_orders, freq="2h")
    datetime_formats = [
        lambda d: d.strftime("%Y-%m-%d %H:%M:%S"),
        lambda d: d.strftime("%Y-%m-%dT%H:%M:%SZ"),
        lambda d: d.strftime("%d/%m/%Y %H:%M"),
        lambda d: "invalid_date",  # intentionally invalid
    ]

    created_at = []
    for i, d in enumerate(base_dates):
        if i % 50 == 0:  # 2% invalid dates
            created_at.append("invalid_date")
        else:
            fmt = datetime_formats[i % 3]
            created_at.append(fmt(d))

    # Generate amounts with some outliers
    amounts = np.random.lognormal(mean=3.5, sigma=0.8, size=n_orders)
    # Add some extreme outliers
    outlier_indices = np.random.choice(n_orders, size=10, replace=False)
    amounts[outlier_indices] = np.random.uniform(5000, 20000, size=10)

    orders = pd.DataFrame({
        "order_id": [f"ORD{i:05d}" for i in range(1, n_orders + 1)],
        "user_id": np.random.choice(users["user_id"].tolist(), n_orders),
        "amount": amounts.round(2),
        "quantity": np.random.randint(1, 10, n_orders),
        "created_at": created_at,
        "status_clean": np.random.choice(statuses, n_orders),
    })

    return orders, users

orders, users = generate_sample_data()
print(f"✅ Generated {len(orders)} orders and {len(users)} users")
print(f"\nOrders sample:\n{orders.head()}")
print(f"\nUsers sample:\n{users.head()}")

---
# Session 1: Datetime Parsing & Time Parts

## Concept: Timestamps vs Strings

A timestamp should be:
- A real datetime type (`datetime64[ns]`, often UTC-aware)
- Sortable by time
- Groupable by month/week/day

A string "looks like time" but behaves like text.

## Helper Function: `parse_datetime`

Use `errors="coerce"` to safely parse datetimes - invalid formats become NaT (missing).

In [ ]:
#@title Define: parse_datetime
_cell("helper_parse_datetime")

def parse_datetime(
    df: pd.DataFrame,
    col: str,
    *,
    utc: bool = True,
) -> pd.DataFrame:
    """
    Parse a timestamp column safely.
    - invalid strings become NA (errors="coerce")
    - utc=True gives timezone-aware timestamps (recommended default)
    """
    dt = pd.to_datetime(df[col], errors="coerce", utc=utc)
    return df.assign(**{col: dt})

print("✅ parse_datetime defined")

## Exercise 1: Parse `created_at`

**Task:**
1. Parse the `created_at` column using `parse_datetime`
2. Count how many values became NaT (missing) after parsing
3. Verify the dtype changed to datetime

In [ ]:
#@title Exercise 1: Parse created_at
_cell("ex1_parse_created_at")

# YOUR CODE HERE
# Hint: Use parse_datetime(orders, "created_at", utc=True)

orders_parsed = parse_datetime(orders.copy(), "created_at", utc=True)

# Check your work
print(f"Original dtype: {orders['created_at'].dtype}")
print(f"Parsed dtype: {orders_parsed['created_at'].dtype}")
print(f"Missing after parse: {orders_parsed['created_at'].isna().sum()}")

In [ ]:
#@title Test Exercise 1
_cell("test_ex1")

@tracked_test("parse_datetime")
def test_parse_datetime():
    """Test that parse_datetime works correctly."""
    # Create test data
    test_df = pd.DataFrame({
        "timestamp": ["2024-01-01 10:00:00", "invalid", "2024-01-02 11:00:00"]
    })

    result = parse_datetime(test_df, "timestamp", utc=True)

    # Test 1: dtype should be datetime
    assert pd.api.types.is_datetime64_any_dtype(result["timestamp"]), \
        "Column should be datetime type"

    # Test 2: invalid should become NaT
    assert result["timestamp"].isna().sum() == 1, \
        "Invalid dates should become NaT"

    # Test 3: valid dates should parse correctly
    assert result["timestamp"].dropna().iloc[0].year == 2024, \
        "Valid dates should parse correctly"

    print("✅ test_parse_datetime passed!")

test_parse_datetime()

## Helper Function: `add_time_parts`

Add common grouping keys from a timestamp column.

In [ ]:
#@title Define: add_time_parts
_cell("helper_add_time_parts")

def add_time_parts(df: pd.DataFrame, ts_col: str) -> pd.DataFrame:
    """Add common time grouping keys (month, day-of-week, hour, etc.)."""
    ts = df[ts_col]
    return df.assign(
        date=ts.dt.date,
        year=ts.dt.year,
        month=ts.dt.to_period("M").astype("string"),
        dow=ts.dt.day_name(),
        hour=ts.dt.hour,
    )

print("✅ add_time_parts defined")

## Exercise 2: Add Time Parts

**Task:**
1. Parse `created_at` first
2. Add time parts using `add_time_parts`
3. Print the first 5 rows showing: `created_at`, `month`, `dow`, `hour`

In [ ]:
#@title Exercise 2: Add Time Parts
_cell("ex2_add_time_parts")

# YOUR CODE HERE
orders_with_time = (
    orders.copy()
    .pipe(parse_datetime, col="created_at", utc=True)
    .pipe(add_time_parts, ts_col="created_at")
)

print(orders_with_time[["created_at", "month", "dow", "hour"]].head(10))

In [ ]:
#@title Test Exercise 2
_cell("test_ex2")

@tracked_test("add_time_parts")
def test_add_time_parts():
    """Test that add_time_parts creates correct columns."""
    test_df = pd.DataFrame({
        "ts": pd.to_datetime(["2024-03-15 14:30:00", "2024-07-20 09:00:00"], utc=True)
    })

    result = add_time_parts(test_df, "ts")

    # Test 1: Required columns exist
    required_cols = ["date", "year", "month", "dow", "hour"]
    for col in required_cols:
        assert col in result.columns, f"Missing column: {col}"

    # Test 2: Values are correct
    assert result["year"].iloc[0] == 2024, "Year should be 2024"
    assert result["hour"].iloc[0] == 14, "Hour should be 14"
    assert result["dow"].iloc[0] == "Friday", "March 15, 2024 is Friday"

    # Test 3: Month format
    assert "2024-03" in result["month"].iloc[0], "Month should be in YYYY-MM format"

    print("✅ test_add_time_parts passed!")

test_add_time_parts()

## Quick Check

**Question:** What does `errors="coerce"` do?

<details>
<summary>Click to reveal answer</summary>

Invalid datetime strings become NaT (missing values) instead of raising an exception.
</details>

---
# Session 2: Outliers & Core Pandas Operations

## Concept: Percentiles

Percentiles help understand the distribution:
- **p50 (median)**: "typical" value
- **p90**: 90% of values are below this
- **p99**: 99% of values are below this (only 1% are above)

## Exercise 3: Compute Percentiles

**Task:**
1. Compute p50, p90, p99 for the `amount` column
2. Identify if there are extreme outliers

In [ ]:
#@title Exercise 3: Compute Percentiles
_cell("ex3_percentiles")

# YOUR CODE HERE
s = orders["amount"].dropna()
percentiles = s.quantile([0.50, 0.90, 0.99])
print("Percentiles for 'amount':")
print(percentiles)
print(f"\nMax value: {s.max():.2f}")
print(f"If p99 is much smaller than max, you have extreme outliers!")

In [ ]:
#@title Test Exercise 3
_cell("test_ex3")

@tracked_test("percentiles")
def test_percentiles():
    """Test understanding of percentiles."""
    test_s = pd.Series([1, 2, 3, 4, 5, 6, 7, 8, 9, 100])  # 100 is an outlier

    p50 = test_s.quantile(0.50)
    p99 = test_s.quantile(0.99)

    # Test 1: Median should be around 5.5
    assert 5 <= p50 <= 6, f"p50 should be between 5 and 6, got {p50}"

    # Test 2: p99 should be close to the outlier
    assert p99 > 50, f"p99 should capture the outlier region, got {p99}"

    # Test 3: Mean vs Median for outlier detection
    mean = test_s.mean()
    assert mean > p50, "Mean should be > median when there are positive outliers"

    print("✅ test_percentiles passed!")

test_percentiles()

## Helper Function: `iqr_bounds`

IQR method for outlier detection:
- Q1 = 25th percentile
- Q3 = 75th percentile
- IQR = Q3 - Q1
- Outliers are outside `[Q1 - 1.5*IQR, Q3 + 1.5*IQR]`

In [ ]:
#@title Define: iqr_bounds
_cell("helper_iqr_bounds")

def iqr_bounds(s: pd.Series, k: float = 1.5) -> tuple[float, float]:
    """Return (lo, hi) IQR bounds for outlier flagging."""
    x = s.dropna()
    q1 = x.quantile(0.25)
    q3 = x.quantile(0.75)
    iqr = q3 - q1
    return float(q1 - k * iqr), float(q3 + k * iqr)

print("✅ iqr_bounds defined")

## Exercise 4: Count Outliers

**Task:**
1. Compute IQR bounds for `amount`
2. Count how many values are outside the bounds
3. What percentage of orders are outliers?

In [ ]:
#@title Exercise 4: Count Outliers
_cell("ex4_count_outliers")

# YOUR CODE HERE
lo, hi = iqr_bounds(orders["amount"], k=1.5)
print(f"IQR bounds: [{lo:.2f}, {hi:.2f}]")

n_outliers = ((orders["amount"] < lo) | (orders["amount"] > hi)).sum()
pct_outliers = n_outliers / len(orders) * 100
print(f"Outliers: {n_outliers} ({pct_outliers:.1f}%)")

In [ ]:
#@title Test Exercise 4
_cell("test_ex4")

@tracked_test("iqr_bounds")
def test_iqr_bounds():
    """Test IQR bounds calculation."""
    # Create data with known quartiles
    test_s = pd.Series([1, 2, 3, 4, 5, 6, 7, 8, 9, 10])

    lo, hi = iqr_bounds(test_s, k=1.5)

    # For this data: Q1=3.25, Q3=7.75, IQR=4.5
    # lo = 3.25 - 1.5*4.5 = -3.5
    # hi = 7.75 + 1.5*4.5 = 14.5

    assert lo < 0, f"Lower bound should be negative, got {lo}"
    assert hi > 10, f"Upper bound should be > 10, got {hi}"

    # All values should be within bounds
    outliers = (test_s < lo) | (test_s > hi)
    assert outliers.sum() == 0, "No outliers expected in this data"

    print("✅ test_iqr_bounds passed!")

test_iqr_bounds()

## Helper Function: `winsorize`

Cap extreme values for visualization (don't delete rows!).

In [ ]:
#@title Define: winsorize
_cell("helper_winsorize")

def winsorize(s: pd.Series, lo: float = 0.01, hi: float = 0.99) -> pd.Series:
    """Cap values to [p_lo, p_hi] (helpful for visualization, not deletion)."""
    x = s.dropna()
    a, b = x.quantile(lo), x.quantile(hi)
    return s.clip(lower=a, upper=b)

print("✅ winsorize defined")

## Exercise 5: Create Winsorized Amount

**Task:**
1. Create a new column `amount_winsor` using the winsorize function
2. Compare min/max of original vs winsorized

In [ ]:
#@title Exercise 5: Create Winsorized Amount
_cell("ex5_winsorize")

# YOUR CODE HERE
orders_w = orders.assign(amount_winsor=winsorize(orders["amount"]))

print("Original amount:")
print(f"  Min: {orders['amount'].min():.2f}, Max: {orders['amount'].max():.2f}")
print(f"\nWinsorized amount (1%-99%):")
print(f"  Min: {orders_w['amount_winsor'].min():.2f}, Max: {orders_w['amount_winsor'].max():.2f}")

In [ ]:
#@title Test Exercise 5
_cell("test_ex5")

@tracked_test("winsorize")
def test_winsorize():
    """Test winsorization."""
    test_s = pd.Series([1, 2, 3, 4, 5, 100, 200, 300])  # extreme outliers

    result = winsorize(test_s, lo=0.1, hi=0.9)

    # Test 1: Same length
    assert len(result) == len(test_s), "Length should not change"

    # Test 2: Extreme values should be capped
    assert result.max() < test_s.max(), "Max should be reduced"

    # Test 3: Middle values should not change
    # (not strictly true due to clipping, but outliers definitely should change)
    assert result.iloc[-1] < 300, "Extreme value 300 should be capped"

    print("✅ test_winsorize passed!")

test_winsorize()

## Core Pandas Pattern 1: Selection with `.loc`

Avoid chained indexing. Use `.loc[row_condition, columns]` for clarity.

## Exercise 6: Filter Paid Orders

**Task:**
1. Filter rows where `status_clean == "paid"`
2. Keep only `user_id` and `amount` columns
3. Count the rows

In [ ]:
#@title Exercise 6: Filter Paid Orders
_cell("ex6_filter_paid")

# YOUR CODE HERE
paid_orders = orders.loc[
    orders["status_clean"] == "paid",
    ["user_id", "amount"]
]
print(f"Paid orders: {len(paid_orders)}")
print(f"Total revenue from paid orders: ${paid_orders['amount'].sum():,.2f}")

In [ ]:
#@title Test Exercise 6
_cell("test_ex6")

@tracked_test("filter_paid")
def test_filter_paid():
    """Test filtering with .loc."""
    test_df = pd.DataFrame({
        "status": ["paid", "refund", "paid", "pending"],
        "amount": [100, 50, 200, 75],
        "user": ["A", "B", "C", "D"]
    })

    result = test_df.loc[test_df["status"] == "paid", ["user", "amount"]]

    # Test 1: Correct number of rows
    assert len(result) == 2, f"Should have 2 paid orders, got {len(result)}"

    # Test 2: Correct columns
    assert list(result.columns) == ["user", "amount"], "Should have user and amount columns"

    # Test 3: Correct values
    assert result["amount"].sum() == 300, "Sum of paid amounts should be 300"

    print("✅ test_filter_paid passed!")

test_filter_paid()

## Core Pandas Pattern 2: Assignment with `.assign`

Use `.assign()` to add columns without modifying the original DataFrame.

## Exercise 7: Add Boolean Column

**Task:**
Add a column `is_refund` that is True when `status_clean == "refund"`.

In [ ]:
#@title Exercise 7: Add Boolean Column
_cell("ex7_add_boolean")

# YOUR CODE HERE
orders_with_refund = orders.assign(
    is_refund=orders["status_clean"] == "refund"
)
print(orders_with_refund["is_refund"].value_counts())

In [ ]:
#@title Test Exercise 7
_cell("test_ex7")

@tracked_test("add_boolean")
def test_add_boolean():
    """Test adding boolean column with .assign."""
    test_df = pd.DataFrame({
        "status": ["paid", "refund", "paid", "refund"]
    })

    result = test_df.assign(is_refund=test_df["status"] == "refund")

    # Test 1: Column exists
    assert "is_refund" in result.columns, "is_refund column should exist"

    # Test 2: Correct dtype
    assert result["is_refund"].dtype == bool, "Should be boolean dtype"

    # Test 3: Correct values
    assert result["is_refund"].sum() == 2, "Should have 2 refunds"

    print("✅ test_add_boolean passed!")

test_add_boolean()

## Core Pandas Pattern 3: GroupBy/Agg

Most analytics tables come from:
- `groupby` → `agg` → `reset_index`

## Exercise 8: Revenue Per User

**Task:**
Compute per-user summary with:
- `n_orders`: count of orders
- `revenue`: sum of amount
- `aov`: average order value (mean of amount)

In [ ]:
#@title Exercise 8: Revenue Per User
_cell("ex8_revenue_per_user")

# YOUR CODE HERE
per_user = (
    orders.groupby("user_id", dropna=False)
    .agg(
        n_orders=("order_id", "size"),
        revenue=("amount", "sum"),
        aov=("amount", "mean"),
    )
    .reset_index()
    .sort_values("revenue", ascending=False)
)
print(f"Top 5 users by revenue:\n{per_user.head()}")

In [ ]:
#@title Test Exercise 8
_cell("test_ex8")

@tracked_test("revenue_per_user")
def test_revenue_per_user():
    """Test groupby aggregation."""
    test_df = pd.DataFrame({
        "user_id": ["A", "A", "B", "B", "B"],
        "order_id": [1, 2, 3, 4, 5],
        "amount": [100, 200, 50, 50, 100]
    })

    result = (
        test_df.groupby("user_id")
        .agg(
            n_orders=("order_id", "size"),
            revenue=("amount", "sum"),
            aov=("amount", "mean"),
        )
        .reset_index()
    )

    # Test 1: Correct number of users
    assert len(result) == 2, "Should have 2 users"

    # Test 2: Correct order counts
    user_a = result[result["user_id"] == "A"].iloc[0]
    assert user_a["n_orders"] == 2, "User A should have 2 orders"

    # Test 3: Correct revenue
    assert user_a["revenue"] == 300, "User A revenue should be 300"

    # Test 4: Correct AOV
    assert user_a["aov"] == 150, "User A AOV should be 150"

    print("✅ test_revenue_per_user passed!")

test_revenue_per_user()

## Exercise 9: Refund Rate by Status

**Task:**
Calculate the refund rate:
- numerator: number of refunds
- denominator: total orders
- rate: refunds / total

In [ ]:
#@title Exercise 9: Refund Rate
_cell("ex9_refund_rate")

# YOUR CODE HERE
refund_summary = (
    orders.assign(is_refund=orders["status_clean"] == "refund")
    .agg(
        total_orders=("order_id", "size"),
        refunds=("is_refund", "sum"),
    )
)
refund_rate = refund_summary["refunds"] / refund_summary["total_orders"]
print(f"Total orders: {refund_summary['total_orders']}")
print(f"Refunds: {refund_summary['refunds']}")
print(f"Refund rate: {refund_rate:.2%}")

In [ ]:
#@title Test Exercise 9
_cell("test_ex9")

@tracked_test("refund_rate")
def test_refund_rate():
    """Test rate calculation."""
    test_df = pd.DataFrame({
        "order_id": range(10),
        "status": ["paid"] * 8 + ["refund"] * 2
    })

    n_refunds = (test_df["status"] == "refund").sum()
    n_total = len(test_df)
    rate = n_refunds / n_total

    # Test 1: Correct refund count
    assert n_refunds == 2, "Should have 2 refunds"

    # Test 2: Correct total
    assert n_total == 10, "Should have 10 orders"

    # Test 3: Correct rate
    assert rate == 0.2, "Refund rate should be 20%"

    print("✅ test_refund_rate passed!")

test_refund_rate()

---
# Session 3: Safe Joins

## Concept: Join Types

- **Left join**: Keep all rows from left table, enrich with right (most common)
- **Inner join**: Keep only matches (can silently drop data!)
- **Outer join**: Keep all rows from both sides

## Helper Function: `safe_left_join`

Use `validate` parameter to prevent join explosions!

In [ ]:
#@title Define: safe_left_join
_cell("helper_safe_left_join")

def safe_left_join(
    left: pd.DataFrame,
    right: pd.DataFrame,
    on: str | list[str],
    *,
    validate: str,
    suffixes: tuple[str, str] = ("", "_r"),
) -> pd.DataFrame:
    """
    Left-join with enforced join cardinality via `validate=`.

    validate options:
    - "many_to_one": left may repeat keys; right must be unique
    - "one_to_one": both sides unique
    - "many_to_many": allowed, but dangerous!
    """
    return left.merge(
        right,
        how="left",
        on=on,
        validate=validate,
        suffixes=suffixes,
    )

print("✅ safe_left_join defined")

## Helper Function: `assert_unique_key`

Always check that lookup tables have unique keys before joining!

In [ ]:
#@title Define: assert_unique_key
_cell("helper_assert_unique_key")

def assert_unique_key(df: pd.DataFrame, key: str | list[str]) -> None:
    """Assert that key column(s) are unique in the DataFrame."""
    if isinstance(key, str):
        key = [key]

    is_unique = not df.duplicated(subset=key).any()
    if not is_unique:
        n_dups = df.duplicated(subset=key).sum()
        raise AssertionError(f"Key {key} is not unique: {n_dups} duplicates found")

print("✅ assert_unique_key defined")

## Exercise 10: Safe Join Orders to Users

**Task:**
1. Check that `user_id` is unique in users table
2. Join orders (left) with users (right) on `user_id`
3. Verify row count didn't change (no join explosion)

In [ ]:
#@title Exercise 10: Safe Join
_cell("ex10_safe_join")

# YOUR CODE HERE

# Step 1: Check uniqueness
assert_unique_key(users, "user_id")
print("✅ user_id is unique in users table")

# Step 2: Join
joined = safe_left_join(
    orders,
    users,
    on="user_id",
    validate="many_to_one",
    suffixes=("", "_user")
)

# Step 3: Verify row count
assert len(joined) == len(orders), "Row count changed - possible join explosion!"
print(f"✅ Row count preserved: {len(orders)} → {len(joined)}")

# Check match rate
match_rate = 1 - joined["country"].isna().mean()
print(f"✅ Country match rate: {match_rate:.1%}")

In [ ]:
#@title Test Exercise 10
_cell("test_ex10")

@tracked_test("safe_join")
def test_safe_join():
    """Test safe left join."""
    # Create test data
    left = pd.DataFrame({
        "id": [1, 2, 3, 1],  # 1 appears twice
        "value": [10, 20, 30, 40]
    })

    right_unique = pd.DataFrame({
        "id": [1, 2, 3],
        "name": ["A", "B", "C"]
    })

    right_duplicate = pd.DataFrame({
        "id": [1, 1, 2, 3],  # 1 appears twice - will cause explosion!
        "name": ["A1", "A2", "B", "C"]
    })

    # Test 1: Valid many_to_one join
    result = safe_left_join(left, right_unique, on="id", validate="many_to_one")
    assert len(result) == len(left), "Row count should be preserved for many_to_one"

    # Test 2: Join explosion should raise error
    try:
        safe_left_join(left, right_duplicate, on="id", validate="many_to_one")
        assert False, "Should have raised an error for duplicate keys"
    except Exception as e:
        assert "MergeError" in type(e).__name__ or "merge" in str(e).lower(), \
            "Should raise merge validation error"

    # Test 3: Columns from right are added
    assert "name" in result.columns, "Right table columns should be added"

    print("✅ test_safe_join passed!")

test_safe_join()

## Exercise 11: Detect dtype Mismatch

**Question:** Which join keys will match?

A) left has `"001"`, right has `"001"` (both strings)
B) left has `"001"`, right has `1` (string vs int)
C) left has `1`, right has `1` (both int)

In [ ]:
#@title Exercise 11: Detect dtype Mismatch
_cell("ex11_dtype_mismatch")

# YOUR CODE HERE - Test each scenario

# Scenario A: string == string
left_a = pd.DataFrame({"id": ["001", "002"], "val": [1, 2]})
right_a = pd.DataFrame({"id": ["001", "003"], "name": ["A", "C"]})
merged_a = left_a.merge(right_a, on="id", how="left")
print("A) string-string match rate:", merged_a["name"].notna().mean())

# Scenario B: string != int
left_b = pd.DataFrame({"id": ["001", "002"], "val": [1, 2]})
right_b = pd.DataFrame({"id": [1, 3], "name": ["A", "C"]})
merged_b = left_b.merge(right_b, on="id", how="left")
print("B) string-int match rate:", merged_b["name"].notna().mean())

# Scenario C: int == int
left_c = pd.DataFrame({"id": [1, 2], "val": [1, 2]})
right_c = pd.DataFrame({"id": [1, 3], "name": ["A", "C"]})
merged_c = left_c.merge(right_c, on="id", how="left")
print("C) int-int match rate:", merged_c["name"].notna().mean())

In [ ]:
#@title Test Exercise 11
_cell("test_ex11")

@tracked_test("dtype_match")
def test_dtype_match():
    """Test dtype matching in joins."""
    left_str = pd.DataFrame({"id": ["001"], "x": [1]})
    right_str = pd.DataFrame({"id": ["001"], "y": [2]})
    right_int = pd.DataFrame({"id": [1], "y": [2]})

    # String-string should match
    result1 = left_str.merge(right_str, on="id", how="left")
    assert result1["y"].notna().all(), "String-string should match"

    # String-int should NOT match (in most cases)
    result2 = left_str.merge(right_int, on="id", how="left")
    assert result2["y"].isna().all(), "String-int should not match"

    print("✅ test_dtype_match passed!")

test_dtype_match()

## Exercise 12: Build Complete Analytics Table

**Task:**
Build a complete analytics-ready table by:
1. Parsing `created_at`
2. Adding time parts
3. Joining with users
4. Adding winsorized amount
5. Adding outlier flag

In [ ]:
#@title Exercise 12: Build Analytics Table
_cell("ex12_analytics_table")

# YOUR CODE HERE

def add_outlier_flag(df: pd.DataFrame, col: str, *, k: float = 1.5) -> pd.DataFrame:
    """Add a boolean flag for outliers based on IQR."""
    lo, hi = iqr_bounds(df[col], k=k)
    return df.assign(**{f"{col}__is_outlier": (df[col] < lo) | (df[col] > hi)})

# Build the analytics table step by step
analytics = (
    orders.copy()
    .pipe(parse_datetime, col="created_at", utc=True)
    .pipe(add_time_parts, ts_col="created_at")
    .pipe(safe_left_join, users, on="user_id", validate="many_to_one", suffixes=("", "_user"))
    .assign(amount_winsor=lambda d: winsorize(d["amount"]))
    .pipe(add_outlier_flag, col="amount", k=1.5)
)

print("Analytics table columns:")
print(analytics.columns.tolist())
print(f"\nShape: {analytics.shape}")
print(f"\nSample:\n{analytics[['order_id', 'user_id', 'country', 'month', 'amount', 'amount_winsor', 'amount__is_outlier']].head()}")

In [ ]:
#@title Test Exercise 12
_cell("test_ex12")

@tracked_test("analytics_table")
def test_analytics_table():
    """Test complete analytics table build."""
    # Create minimal test data
    test_orders = pd.DataFrame({
        "order_id": ["O1", "O2", "O3"],
        "user_id": ["U1", "U2", "U1"],
        "amount": [100.0, 200.0, 5000.0],  # 5000 is an outlier
        "quantity": [1, 2, 1],
        "created_at": ["2024-01-15 10:00:00", "2024-01-16 11:00:00", "2024-01-17 12:00:00"],
        "status_clean": ["paid", "paid", "refund"]
    })

    test_users = pd.DataFrame({
        "user_id": ["U1", "U2"],
        "country": ["USA", "UK"],
        "signup_date": ["2024-01-01", "2024-01-02"]
    })

    result = (
        test_orders
        .pipe(parse_datetime, col="created_at", utc=True)
        .pipe(add_time_parts, ts_col="created_at")
        .pipe(safe_left_join, test_users, on="user_id", validate="many_to_one", suffixes=("", "_user"))
        .assign(amount_winsor=lambda d: winsorize(d["amount"]))
        .pipe(add_outlier_flag, col="amount", k=1.5)
    )

    # Test 1: Row count preserved
    assert len(result) == 3, "Should have 3 rows"

    # Test 2: Time parts exist
    assert all(col in result.columns for col in ["month", "dow", "hour"]), \
        "Time parts should exist"

    # Test 3: User data joined
    assert "country" in result.columns, "Country should be joined"
    assert result["country"].notna().all(), "All users should match"

    # Test 4: Outlier flag exists
    assert "amount__is_outlier" in result.columns, "Outlier flag should exist"

    # Test 5: Outlier detected
    assert result["amount__is_outlier"].sum() >= 1, "Should detect at least 1 outlier"

    print("✅ test_analytics_table passed!")

test_analytics_table()

---
# Summary & Final Verification

## Key Takeaways

1. **Datetime Parsing**: Always use `errors="coerce"` to handle invalid dates safely
2. **Time Parts**: Add grouping keys (month, dow, hour) for easy aggregation
3. **Outliers**: Use IQR bounds to flag, winsorize for visualization (don't delete!)
4. **Core Pandas**: `.loc`, `.assign`, `groupby/agg` are your daily workhorses
5. **Safe Joins**: Use `validate=` to prevent explosions, check uniqueness first

In [ ]:
#@title Run All Tests
_cell("final_verification")

# Run all tests to verify your understanding
print("=" * 60)
print("RUNNING ALL UNIT TESTS")
print("=" * 60)

tests = [
    ("parse_datetime", test_parse_datetime),
    ("add_time_parts", test_add_time_parts),
    ("percentiles", test_percentiles),
    ("iqr_bounds", test_iqr_bounds),
    ("winsorize", test_winsorize),
    ("filter_paid", test_filter_paid),
    ("add_boolean", test_add_boolean),
    ("revenue_per_user", test_revenue_per_user),
    ("refund_rate", test_refund_rate),
    ("safe_join", test_safe_join),
    ("dtype_match", test_dtype_match),
    ("analytics_table", test_analytics_table),
]

passed = 0
failed = 0

for name, test_func in tests:
    try:
        test_func()
        passed += 1
    except Exception as e:
        print(f"❌ {name} FAILED: {e}")
        failed += 1

print()
print("=" * 60)
print(f"RESULTS: {passed} passed, {failed} failed")
print("=" * 60)

if failed == 0:
    print("\n🎉 Congratulations! All tests passed!")
else:
    print(f"\n⚠️ {failed} test(s) need attention. Review the exercises above.")

In [ ]:
#@title Final Summary
_cell("final_summary")

print("=" * 60)
print("FINAL VERIFICATION")
print("=" * 60)
print()
print(f"Student: {STUDENT_NAME}")
print(f"Session: {_SESS}")
print(f"Cells run: {_cell_num}")
print(f"Exercises attempted: {len(_attempts)}")
if _attempts:
    for ex, tries in sorted(_attempts.items()):
        print(f"   - {ex}: {tries} attempt(s)")
print()
_capture("summary", {"cells": _cell_num, "exercises": len(_attempts)})
print("Great work!")
print("=" * 60)